In [1]:
#### In this notebook I obtain the branching ratio for the processes zp -> chi chi, 
#### y0 -> chi chi, and h2 -> chi chi, and store the results in a dataframe that
#### contains the mediator mass, and the branching ratios

import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import scipy
import matplotlib as mpl
import sympy as sp
from matplotlib.colors import LogNorm



plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

In [2]:
BRdata = pd.DataFrame(columns=['Mediator', '$m_{med}$', '$g_{q}$', '$\sin\\theta$', 
                               '$g_{\chi}$', '$y_{\chi}$', '$BR(med>\chi\chi)$'])

In [3]:
## spin 1 mediator ##

# mediator mass
MZp = np.arange(140, 3010, 10)

# quarks masses
MU = 0.00255
MC = 1.27
MT = 172
MD = 0.00504
MS = 0.101
MB = 4.7

# DM mass
Mchi = 65.0

# couplings
gqV = np.array([0.01, 0.1, 0.2, 0.25])
gchi = np.array([0.1, 0.5, 1.0, 2.0])
gqA = 0.0
gZp = 1.0


In [4]:
for gq in gqV:
    for g in gchi:
        for m in MZp:
            gqL = (gqA + gq)/2.
            gqR = (-gqA + gq)/2.
            
            # Zp to t 
            if m < 2*MT:
                ZPtt = 0
            else:
                
                ZPtt = ((-6*gqL**2*gZp**2*MT**2 + 36*gqL*gqR*gZp**2*MT**2 - 6*gqR**2*gZp**2*MT**2 
                         + 6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2)
                        *np.sqrt(-4*MT**2*m**2 + m**4))/(48.*np.pi*abs(m)**3)            
            
            # Zp to u
            ZPuu = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

            # Zp to c
            ZPcc = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

            # Zp to d
            ZPdd = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

            # Zp to s
            ZPss = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

            # Zp to b
            ZPbb = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

            # Zp to DM
            ZPchichi = ((-16*g**2*Mchi**2 + 4*g**2*m**2)
                        *np.sqrt(-4*Mchi**2*m**2 + m**4))/(96.*np.pi*abs(m)**3)

            # Zp total width
            GammaZP = ZPuu + ZPcc + ZPtt + ZPdd + ZPss + ZPbb + ZPchichi

            # BR(Zp to DM)
            BR_ZPchichi = ZPchichi / GammaZP

            # BR(Zp to quarks)
            BR_ZPqq = (ZPuu + ZPcc + ZPtt + ZPdd + ZPss + ZPbb)/GammaZP
            
            BRdata.loc[len(BRdata)] = ['$Z^{\prime}$', m, gq, np.nan, g, np.nan, BR_ZPchichi]

In [5]:
BRdata

,Mediator,$m_{med}$,$g_{q}$,$\sin\theta$,$g_{\chi}$,$y_{\chi}$,$BR(med>\chi\chi)$
0,$Z^{\prime}$,140,0.01,NaN,0.1,NaN,0.405367
1,$Z^{\prime}$,150,0.01,NaN,0.1,NaN,0.623433
2,$Z^{\prime}$,160,0.01,NaN,0.1,NaN,0.725391
3,$Z^{\prime}$,170,0.01,NaN,0.1,NaN,0.781062
4,$Z^{\prime}$,180,0.01,NaN,0.1,NaN,0.815220
...,...,...,...,...,...,...,...
4587,$Z^{\prime}$,2960,0.25,NaN,2.0,NaN,0.876400
4588,$Z^{\prime}$,2970,0.25,NaN,2.0,NaN,0.876402
4589,$Z^{\prime}$,2980,0.25,NaN,2.0,NaN,0.876404
4590,$Z^{\prime}$,2990,0.25,NaN,2.0,NaN,0.876406


In [6]:
## spin-0 mediator ##

# mediator mass
MSd = np.arange(140, 1510, 10)

# quarks masses
MT = 172

# tau lepton mass
MTA = 1.78

# DM mass
Mchi = 65.0

# gauge bosons masses
MW = 80.4
MZ = 91.2
MZp = 3000.0

# higgs boson mass
MH = 125.3

# vevs
vev = 246
gchi = 1.0
vev2 = MZp/(2*gchi)

# couplings
ychi = np.array([0.1, 0.5, 1.0, 2.0])
yt = np.sqrt(2)*MT/vev2
ytau = np.sqrt(2)*MTA/vev2
cw = MW/MZ
sw = np.sqrt(1 - cw**2)
ee = (2*MW*sw)/vev
Sa = np.array([0.1, 0.2, 0.3])


In [7]:
for s in Sa:
    for y in ychi:
        for m in MSd:
            Ca = np.sqrt(1-s**2)
            lam1 = (Ca**2*MH**2)/(2.*vev**2) + (m**2*s**2)/(2.*vev**2)
            lam2 = (Ca**2*m**2)/(2.*vev2**2) + (MH**2*s**2)/(2.*vev2**2)
            lam3 = (Ca*(-MH**2 + m**2)*s)/(vev*vev2)

            # S to Higgs
            if m < 2* MH:
                Shh = 0
            else:
                Shh = ((36*Ca**4*lam1**2*s**2*vev**2 - 24*Ca**4*lam1*lam3*s**2*vev**2 + 4*Ca**4*lam3**2*s**2*vev**2 
                        + 12*Ca**2*lam1*lam3*s**4*vev**2 - 4*Ca**2*lam3**2*s**4*vev**2 + lam3**2*s**6*vev**2 
                        + 12*Ca**5*lam1*lam3*s*vev*vev2 - 4*Ca**5*lam3**2*s*vev*vev2 + 72*Ca**3*lam1*lam2*s**3*vev*vev2 
                        - 24*Ca**3*lam1*lam3*s**3*vev*vev2 - 24*Ca**3*lam2*lam3*s**3*vev*vev2 + 10*Ca**3*lam3**2*s**3*vev*vev2 
                        + 12*Ca*lam2*lam3*s**5*vev*vev2 - 4*Ca*lam3**2*s**5*vev*vev2 + Ca**6*lam3**2*vev2**2 
                        + 12*Ca**4*lam2*lam3*s**2*vev2**2 - 4*Ca**4*lam3**2*s**2*vev2**2 + 36*Ca**2*lam2**2*s**4*vev2**2 
                        - 24*Ca**2*lam2*lam3*s**4*vev2**2 + 4*Ca**2*lam3**2*s**4*vev2**2)
                       *np.sqrt(-4*MH**2*m**2 + m**4))/(32.*np.pi*abs(m)**3)

            # S to DM
            Schichi = ((-4*Ca**2*Mchi**2*y**2 + Ca**2*m**2*y**2)
                       *np.sqrt(-4*Mchi**2*m**2 + m**4))/(32.*np.pi*abs(m)**3)

            # S to tau
            Stautau = ((m**2*s**2*ytau**2 - 4*MTA**2*s**2*ytau**2)
                       *np.sqrt(m**4 - 4*m**2*MTA**2))/(16.*np.pi*abs(m)**3)

            # S to top
            if m < 2*MT:
                Stt = 0
            else:
                Stt = ((3*m**2*s**2*yt**2 - 12*MT**2*s**2*yt**2)
                       *np.sqrt(m**4 - 4*m**2*MT**2))/(16.*np.pi*abs(m)**3)

            # S to W
            if m < 2*MW:
                Sww = 0
            else:
                Sww = (((3*ee**4*s**2*vev**2)/(4.*sw**4) + (ee**4*m**4*s**2*vev**2)/(16.*MW**4*sw**4) 
                        - (ee**4*m**2*s**2*vev**2)/(4.*MW**2*sw**4))
                       *np.sqrt(m**4 - 4*m**2*MW**2))/(16.*np.pi*abs(m)**3)

            # S to Z
            if m < 2*MZ:
                Szz = 0
            else:
                Szz = (((9*ee**4*s**2*vev**2)/2. + (3*ee**4*m**4*s**2*vev**2)
                        /(8.*MZ**4) - (3*ee**4*m**2*s**2*vev**2)/(2.*MZ**2) 
                        + (3*cw**4*ee**4*s**2*vev**2)/(4.*sw**4) + (cw**4*ee**4*m**4*s**2*vev**2)
                        /(16.*MZ**4*sw**4) - (cw**4*ee**4*m**2*s**2*vev**2)
                        /(4.*MZ**2*sw**4) + (3*cw**2*ee**4*s**2*vev**2)/sw**2 
                        + (cw**2*ee**4*m**4*s**2*vev**2)/(4.*MZ**4*sw**2) 
                        - (cw**2*ee**4*m**2*s**2*vev**2)/(MZ**2*sw**2) + (3*ee**4*s**2*sw**2*vev**2)/cw**2 
                        + (ee**4*m**4*s**2*sw**2*vev**2)/(4.*cw**2*MZ**4) - (ee**4*m**2*s**2*sw**2*vev**2)/(cw**2*MZ**2) 
                        + (3*ee**4*s**2*sw**4*vev**2)/(4.*cw**4) + (ee**4*m**4*s**2*sw**4*vev**2)/(16.*cw**4*MZ**4) 
                        - (ee**4*m**2*s**2*sw**4*vev**2)/(4.*cw**4*MZ**2))
                       *np.sqrt(m**4 - 4*m**2*MZ**2))/(32.*np.pi*abs(m)**3)


            #  S Total width
            GammaS = Shh + Schichi + Stautau + Stt + Sww + Szz

            # BR(S to DM)
            BR_Schichi = Schichi / GammaS
            
            BRdata.loc[len(BRdata)] = ['$S_{d}$', m, np.nan, s, np.nan, y, BR_Schichi]

In [8]:
BRdata

,Mediator,$m_{med}$,$g_{q}$,$\sin\theta$,$g_{\chi}$,$y_{\chi}$,$BR(med>\chi\chi)$
0,$Z^{\prime}$,140,0.01,NaN,0.1,NaN,0.405367
1,$Z^{\prime}$,150,0.01,NaN,0.1,NaN,0.623433
2,$Z^{\prime}$,160,0.01,NaN,0.1,NaN,0.725391
3,$Z^{\prime}$,170,0.01,NaN,0.1,NaN,0.781062
4,$Z^{\prime}$,180,0.01,NaN,0.1,NaN,0.815220
...,...,...,...,...,...,...,...
6231,$S_{d}$,1460,NaN,0.3,NaN,2.0,0.226537
6232,$S_{d}$,1470,NaN,0.3,NaN,2.0,0.224156
6233,$S_{d}$,1480,NaN,0.3,NaN,2.0,0.221810
6234,$S_{d}$,1490,NaN,0.3,NaN,2.0,0.219496


In [9]:
BRdata.to_pickle('../data/BRdata.pcl')